In [1]:
# Dataset management packages
from spivutils.synthetic_datasets.spid import load_data
from spivutils.batch_generators.keras_generator import batch_data
from spivutils.common_tools.pre_processors import preprocess_data
from spivutils.common_tools.operations import normalization, vectoraddition, thresholding

# Model packages
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Reshape, Activation

In [2]:
# Force CPU usage
tf.config.set_visible_devices([], 'GPU')

In [3]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data()


 Checking files ...

 train_x.npy  Found!

 train_y.npy  Found!

 valid_x.npy  Found!

 valid_y.npy  Found!

 test_x.npy  Found!

 test_y.npy  Found!

 Importing data ...

 Data loading successful!


In [4]:
batch_size = 10
chunk_size = 1000

# Collect a chunk of the dataset
train_x_chunk = train_x[0:chunk_size]
train_y_chunk = train_y[0:chunk_size]
valid_x_chunk = valid_x[0:chunk_size]
valid_y_chunk = valid_y[0:chunk_size]

# Load data in batches to avoid memory overload
train_batch = batch_data(train_x_chunk, train_y_chunk, batch_size)
valid_batch = batch_data(valid_x_chunk, valid_y_chunk, batch_size)

Pre-processing

In [5]:
# Add pre-processing operations
train_batch.add_x_preprocessing_operation(thresholding)
train_batch.add_x_preprocessing_operation(normalization)
train_batch.add_y_preprocessing_operation(vectoraddition)

# Add pre-processing operations
valid_batch.add_x_preprocessing_operation(thresholding)
valid_batch.add_x_preprocessing_operation(normalization)
valid_batch.add_y_preprocessing_operation(vectoraddition)

In [6]:
input_shape = train_batch[0][0][0,].shape
output_shape = train_batch[0][1][0,].shape

# Model hyperparameters
filters = 32
kernel_size = (1,10,10)
pool_size = (1,5,5)

In [7]:
model = Sequential()

model.add(Conv3D(filters, kernel_size = kernel_size, padding = 'same', input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size = pool_size))

model.add(Conv3D(filters, kernel_size = kernel_size, padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size = pool_size))

model.add(Conv3D(filters, kernel_size = kernel_size, padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size = pool_size))

model.add(Flatten())

model.add(Dense(filters))
model.add(Activation('relu'))

model.add(Dense(filters))
model.add(Activation('linear'))

model.add(Dense(np.prod(output_shape)))

model.add(Reshape(output_shape))

model.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 2, 630, 665, 32)   3232      
                                                                 
 activation (Activation)     (None, 2, 630, 665, 32)   0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 2, 126, 133, 32)  0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 2, 126, 133, 32)   102432    
                                                                 
 activation_1 (Activation)   (None, 2, 126, 133, 32)   0         
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 2, 25, 26, 32)    0         
 3D)                                                    

In [9]:
model.fit(train_batch, validation_data = valid_batch, epochs = 10)

Epoch 1/10
100/100 [==============================] - 3356s 34s/step - loss: 7.0091 - val_loss: 5.3157
Epoch 2/10
 92/100 [==========================>...] - ETA: 4:09 - loss: 5.4419

KeyboardInterrupt: 